
# 📘 UC22 – Aula 6 — Filtros Compostos (Pandas)

![Python](https://img.shields.io/badge/Python-3.11+-blue?logo=python)
![Pandas](https://img.shields.io/badge/Pandas-Filtros%20Compostos-green?logo=pandas)
![Google Colab](https://img.shields.io/badge/Google%20Colab-Notebook-yellow?logo=googlecolab)
![Tempo](https://img.shields.io/badge/Dura%C3%A7%C3%A3o-90%20min-red)
![N%C3%ADvel](https://img.shields.io/badge/N%C3%ADvel-Iniciante%E2%9E%9CIntermedi%C3%81rio-purple)

**Turma:** 3º Ano – Ensino Médio Técnico em Informática  
**Tema:** Combinação de condições (E/OU/NÃO) para filtrar linhas em DataFrames  
**Ferramenta principal:** Google Colab

> “Filtros compostos são como estratégias de batalha: o segredo está na **combinação**.” – IAra ⚔️



## 🎯 Objetivos de Aprendizagem
- Combinar condições com **AND (`&`)**, **OR (`|`)** e **NOT (`~`)** no Pandas.  
- Usar **parênteses** corretamente (ordem de avaliação).  
- Criar filtros robustos com `isin`, `between`, `str.contains` (com `na=False`).  
- Alternar entre sintaxe padrão e **`.query()`** (com variáveis via `@`).  
- Montar **máscaras booleanas** reutilizáveis (código claro e legível).



## 📂 Preparação do Ambiente (Google Drive)
Usaremos os mesmos arquivos das aulas anteriores, mantidos em **`Meu Drive/UC22/`**:

- `pokemons_pokedex.csv`  
- `escolas_estaduais_censo_escolar_2023.csv`


In [ ]:

# 📂 Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 🐍 Importações essenciais
import pandas as pd

# 🗂️ Definição dos caminhos (ajuste conforme sua pasta)
caminho_poke = "/content/drive/MyDrive/UC22/pokemons_pokedex.csv"
caminho_escolas = "/content/drive/MyDrive/UC22/escolas_estaduais_censo_escolar_2023.csv"

# 📥 Leitura dos datasets com codificação apropriada
df_poke = pd.read_csv(caminho_poke, encoding="utf-8")
df_escolas = pd.read_csv(caminho_escolas, encoding="latin-1", sep=",")

# 🧹 Padronização de nomes de colunas (minúsculas, sem espaços)
df_poke.columns    = df_poke.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)
df_escolas.columns = df_escolas.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)

# 🔤 Padronização opcional para facilitar filtros de texto
if "no_municipio" in df_escolas.columns:
    df_escolas["no_municipio"] = df_escolas["no_municipio"].astype(str).strip().str.upper()

print("✅ Pokémon:", df_poke.shape, " | colunas:", len(df_poke.columns))
print("✅ Escolas:", df_escolas.shape, " | colunas:", len(df_escolas.columns))

# 🔎 Visualize as primeiras colunas para confirmar nomes
list(df_poke.columns)[:15]



## 🧠 Revisão Rápida (Aula 5 → Aula 6)
- Filtro **simples**: uma condição (ex.: `df['attack'] > 100`).  
- Filtros **compostos**: **duas ou mais condições** combinadas (ex.: ataque alto **E** velocidade alta).  
- Após a leitura, **padronizamos os nomes de colunas** para minúsculas e com `_`.



## 1) Operadores lógicos no Pandas (entre *Series* booleanas)
- **AND:** `cond1 & cond2` → ambas verdadeiras  
- **OR:** `cond1 | cond2` → pelo menos uma verdadeira  
- **NOT:** `~cond` → inverte True/False

⚠️ **Importante:** Sempre use **parênteses** envolvendo cada condição!


In [ ]:

# 🎯 Exemplo 1 — Água E rápida E defensiva moderada
# Objetivo: Pokémon do tipo Água com Speed > 70 E Defense >= 60
cond_tipo   = df_poke['type_1'] == 'Water'
cond_speed  = df_poke['speed'] > 70
cond_def    = df_poke['defense'] >= 60

mask = cond_tipo & cond_speed & cond_def  # AND entre as condições
agua_rapida_defensiva = df_poke[mask][['name','type_1','defense','speed','attack']].copy()

print("Qtd Água & Speed>70 & Defense>=60:", agua_rapida_defensiva.shape[0])
agua_rapida_defensiva.head(10)


In [ ]:

# 🎯 Exemplo 2 — Fogo OU Gelo com ataque alto (>100)
cond_tipo  = (df_poke['type_1'] == 'Fire') | (df_poke['type_1'] == 'Ice')
cond_atk   = df_poke['attack'] > 100

mask = cond_tipo & cond_atk  # (Fogo OU Gelo) E ataque alto
fogo_ou_gelo_fortes = df_poke[mask][['name','type_1','attack','speed']].copy()
fogo_ou_gelo_fortes.head(10)


In [ ]:

# 🎯 Exemplo 3 — NÃO lendário e muito veloz (Speed >= 120)
col_leg = 'legendary' if 'legendary' in df_poke.columns else None

if col_leg:
    mask = (~df_poke[col_leg]) & (df_poke['speed'] >= 120)
    nao_lendarios_rapidos = df_poke[mask][['name','type_1','speed', col_leg]].copy()
    nao_lendarios_rapidos.head(10)
else:
    print("⚠️ Coluna 'legendary' não encontrada. Verifique df_poke.columns")



## 2) Funções úteis para compor filtros
### 2.1 `isin` — pertence a uma lista


In [ ]:

# 🎯 Tipos alvo: Electric, Psychic, Dragon
alvos = ['Electric','Psychic','Dragon']
mask  = df_poke['type_1'].isin(alvos)
df_poke[mask][['name','type_1','attack','speed']].head(10)



### 2.2 `between` — intervalo fechado [min, max]


In [ ]:

# 🎯 Attack entre 80 e 100 (inclusive), e Speed >= 80
mask = df_poke['attack'].between(80, 100) & (df_poke['speed'] >= 80)
df_poke[mask][['name','type_1','attack','speed']].head(10)



### 2.3 `str.contains` — filtro de texto (com segurança contra NaN)


In [ ]:

# 🎯 Nomes que contêm 'mega' (qualquer caso), sem quebrar em valores NaN
mask_nome = df_poke['name'].astype(str).str.contains('mega', case=False, na=False)
df_poke[mask_nome][['name','type_1','attack','defense','speed']].head(10)



## 3) `.query()` como alternativa mais legível
- Escreva condições como texto, semelhante a SQL.  
- Use `@variavel` para passar listas/valores Python.


In [ ]:

# 🎯 Exemplo com .query(): tipo em lista e ataque alto
tipos    = ['Fire','Ice']
ataque_corte = 100

resultado = df_poke.query("type_1 in @tipos and attack > @ataque_corte")[['name','type_1','attack','speed']].copy()
resultado.head(10)



## 4) Máscaras booleanas reutilizáveis (código limpo e legível)


In [ ]:

# 🎯 Time ofensivo: Attack > 110 E Speed > 95, NÃO lendário
col_leg = 'legendary' if 'legendary' in df_poke.columns else None

cond_ofensivo = (df_poke['attack'] > 110) & (df_poke['speed'] > 95)
cond_nao_leg  = ~df_poke[col_leg] if col_leg else True

mask_time = cond_ofensivo & cond_nao_leg
time_ofensivo = df_poke[mask_time].copy()

# Ordenamos para facilitar a escolha final
time_ofensivo = time_ofensivo.sort_values(by=['attack','speed'], ascending=False)
time_ofensivo[['name','type_1','attack','speed']].head(15)



## 5) Aplicando filtros compostos no dataset de Escolas (texto + OU/AND)


In [ ]:

# 🎯 Ex.: escolas em CAMPINAS OU SANTOS, com dependência ESTADUAL
if {'no_municipio','tp_dependencia'}.issubset(df_escolas.columns):
    cidades = ['CAMPINAS','SANTOS']
    mask_cidade = df_escolas['no_municipio'].isin(cidades)
    mask_dep    = df_escolas['tp_dependencia'].astype(str).str.upper().str.contains('ESTADUAL', na=False)

    escolas_alvo = df_escolas[mask_cidade & mask_dep][['no_entidade','no_municipio','tp_dependencia']].copy()
    print("Qtd escolas (Campinas ou Santos, Estadual):", escolas_alvo.shape[0])
    escolas_alvo.head(10)
else:
    print("⚠️ Verifique as colunas: no_municipio / tp_dependencia")



## ✅ Mini-Lab (entregar no Classroom)
**Arquivo:** `UC22_Aula06_Pratica_SeuNome.ipynb`

### Parte 1 — Pokémon
1. **Defensores Ágeis:** Liste Pokémon com **defense ≥ 100** **E** **speed ≥ 90**. Mostre `name`, `type_1`, `defense`, `speed`, **ordenando** por `defense` (desc) e depois `speed` (desc).  
2. **Ataque “na régua”:** Selecione Pokémon com `attack` **entre 80 e 100** (inclusive) **OU** que sejam do tipo **Electric**. Mostre `name`, `type_1`, `attack`.  
3. **Filtro de texto seguro:** Encontre Pokémon cujo `name` contenha “mega” (qualquer caso) **E** `attack > 100`. Mostre `name`, `type_1`, `attack`.  
4. **Listas de tipos:** Usando `isin`, filtre Pokémon cujo `type_1` pertença a **['Water','Grass','Ground']** **E** `hp ≥ 80`. Mostre `name`, `type_1`, `hp`.

### Parte 2 — Escolas
1. Selecione escolas em **CAMPINAS OU SANTOS** **E** com `tp_dependencia` contendo **“ESTADUAL”** (maiúsculas). Mostre `no_entidade`, `no_municipio`, `tp_dependencia`.  
2. Se existir a coluna de **código** (ex.: `co_entidade`), filtre escolas com código **entre 35000000 e 35099999** **E** `no_municipio == 'SÃO PAULO'`. Mostre `co_entidade`, `no_entidade`, `no_municipio`.

> **Dica:** Crie **máscaras nomeadas** (ex.: `mask_cidade`, `mask_dep`) para deixar o código claro.



### 📝 Espaço para suas respostas (execute os blocos e comente seus achados)


In [ ]:

# 🔧 Resposta Pokémon 1 — Defensores Ágeis
mask = (df_poke['defense'] >= 100) & (df_poke['speed'] >= 90)
resp1 = df_poke[mask][['name','type_1','defense','speed']].copy().sort_values(['defense','speed'], ascending=[False, False])
print(resp1.shape)
resp1.head(15)


In [ ]:

# 🔧 Resposta Pokémon 2 — Attack 80–100 OU Electric
mask = df_poke['attack'].between(80, 100) | (df_poke['type_1'] == 'Electric')
resp2 = df_poke[mask][['name','type_1','attack']].copy()
print(resp2.shape)
resp2.head(15)


In [ ]:

# 🔧 Resposta Pokémon 3 — name contém 'mega' E attack > 100
mask = df_poke['name'].astype(str).str.contains('mega', case=False, na=False) & (df_poke['attack'] > 100)
resp3 = df_poke[mask][['name','type_1','attack']].copy()
print(resp3.shape)
resp3.head(15)


In [ ]:

# 🔧 Resposta Pokémon 4 — tipos em lista E hp ≥ 80
mask = df_poke['type_1'].isin(['Water','Grass','Ground']) & (df_poke['hp'] >= 80)
resp4 = df_poke[mask][['name','type_1','hp']].copy()
print(resp4.shape)
resp4.head(15)


In [ ]:

# 🔧 Resposta Escolas 1 — CAMPINAS OU SANTOS + Estadual
if {'no_municipio','tp_dependencia','no_entidade'}.issubset(df_escolas.columns):
    mask_cidade = df_escolas['no_municipio'].isin(['CAMPINAS','SANTOS'])
    mask_dep = df_escolas['tp_dependencia'].astype(str).str.upper().str.contains('ESTADUAL', na=False)
    resp_escolas1 = df_escolas[mask_cidade & mask_dep][['no_entidade','no_municipio','tp_dependencia']].copy()
    print(resp_escolas1.shape)
    resp_escolas1.head(15)
else:
    print("Verifique colunas: no_municipio, tp_dependencia, no_entidade")


In [ ]:

# 🔧 Resposta Escolas 2 — Código por faixa E município SÃO PAULO (se existir co_entidade)
if {'co_entidade','no_municipio','no_entidade'}.issubset(df_escolas.columns):
    mask_codigo = df_escolas['co_entidade'].between(35000000, 35099999)
    mask_mun = df_escolas['no_municipio'] == 'SÃO PAULO'
    resp_escolas2 = df_escolas[mask_codigo & mask_mun][['co_entidade','no_entidade','no_municipio']].copy()
    print(resp_escolas2.shape)
    resp_escolas2.head(15)
else:
    print("Colunas necessárias não encontradas. Use df_escolas.columns para investigar.")



## 🧯 Erros Comuns e Dicas
- **Faltam parênteses:** `cond1 & cond2` → **sempre** `(cond1) & (cond2)`.  
- **`and`/`or`/`not`:** em Pandas, use `&`, `|`, `~` (os do Python lógico não funcionam entre Series).  
- **`KeyError`**: nome de coluna errado → confira `df.columns`.  
- **`str.contains` com NaN:** use `na=False` e `astype(str)` para evitar erros.  
- **Tipos mistos:** ao comparar números, garanta tipo numérico com `pd.to_numeric()` se necessário.  
- **Filtro vazio:** não é erro; significa “nenhuma linha atende às condições”.



## 📎 Conclusão
Você aprendeu a **combinar condições** para criar filtros poderosos e legíveis.  
Na **Aula 7**, vamos para **Agrupamentos com `groupby()`** — médias por tipo, contagens por categoria, top-N por grupo… o coração das análises comparativas!



---
*Notebook gerado em: 2025-09-12 12:59:02*
